In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Set the EXTRACT, TRANSFORM, LOAD directories
# in a default environment setup, these would be relative to this .ipynb (or .py if this notebook has been converted) 

dir_extract="./01.DataExtract/"
dir_transform="./02.DataTransform/"
dir_load="./03.DataLoad/"


### Test File = "test_contacts.csv"

- mark - multiple (4) identical records
- michael - multiple (3) indentical records
- kevin - 3 unique records; 2 with same Contact ID but different email2 values, 1 with unique Contact ID but rest of data identical to one of the other records.
- maggie - 2 records, 2nd record have different Primary Email and email2 values
- herb - 2 records, 2nd record has a blank Primary Email field
- sydney - 2 records, 2nd record has blank Contact ID
- jeff - 1 unique record with blank Contact ID

### KR file = "KR_Contacts_Names_Emails_00000_80000.csv"

In [ ]:
# Set input filenames

filein_allRecords = "KR_Contacts_Names_Emails_00000_80000.csv"

In [ ]:
# Set output filenames

# initial sets of duplicate records
fileout_duplicates = (
    "out_" +
    filein_allRecords.replace(".csv","").replace("_","") +
    "_duplicates.csv")
print(fileout_duplicates)

# working file with initial sets of duplicate records removed
fileout_workingStep01 = (
    "out_" +
    filein_allRecords.replace(".csv","").replace("_","") +
    "_workingStep01.csv")
print(fileout_workingStep01)

# working file with de-duped records added back in
fileout_workingStep02 = (
    "out_" +
    filein_allRecords.replace(".csv","").replace("_","") +
    "_workingStep02.csv")
print(fileout_workingStep02)

# records for staff manual cleanup of CiviCRM - records with Duplicate Contact IDs
fileout_manualCleanupDupeOnContactID = (
    "out_" +
    filein_allRecords.replace(".csv","").replace("_","") +
    "_manualCleanupDupeOnContactID.csv")
print(fileout_manualCleanupDupeOnContactID)

# records for staff manual cleanup of CiviCRM - records with Duplicate Primary Emails (different Contact IDs)
fileout_manualCleanupDupeOnEmail = (
    "out_" +
    filein_allRecords.replace(".csv","").replace("_","") +
    "_manualCleanupDupeOnEmail.csv")
print(fileout_manualCleanupDupeOnEmail)

# records for staff manual cleanup of CiviCRM - records with blank Contact IDs - something has gone horribly wrong
fileout_manualCleanupBlankContactID = (
    "out_" +
    filein_allRecords.replace(".csv","").replace("_","") +
    "_manualCleanupBlankContactID.csv")
print(fileout_manualCleanupBlankContactID)

In [ ]:
# load the initial extract file into dataframe
df_extract = pd.read_csv(dir_extract + filein_allRecords)

# create a working dataframe
df_working = df_extract

print(df_working)

In [ ]:
# Collect duplicates for further processing
mask_duplicateRows = df_working.duplicated(keep=False)

print(f"\n{mask_duplicateRows}")

df_duplicates = df_working[mask_duplicateRows].sort_values(by='Contact ID')
print(f"\n{df_duplicates}")

# Write duplicates to a file
df_duplicates.to_csv(dir_transform + fileout_duplicates, index=False)

### !!! in KR's file, this produces 421 rows (odd!)
this means there are sets where there an odd number of duplicate records
in this case, example is

| Contact | No. | Name |
| -- | -- | -- |
| 2063 | 3 | Julie Gates
| 6340 | 3 | Cliff Mewdell
| 9501 | 3 | Josh Shook
| 12610 | 3 | Sage Walker
| 13141 | 3 | Michael Rehak
| 18413 | 3 | Damian Fox
| 19423 | 4 | marco tejada
| 21086 | 4 | Lindsay Hacker
| 28647 | 4 | Stephen Moore
| 31793 | 3 | Hyedie Hashimoto


# SCRATCH

# Count the number of occurrences of each value in the 'Contact ID' column
counts = df_duplicates.groupby('Contact ID')['Contact ID'].transform('count')

# Create a mask for rows where 'Contact ID' occurs exactly 3 times
mask = counts == 3

# Apply the mask to the DataFrame
df_filtered = df_duplicates[mask]

print(df_filtered)


# Create a mask for rows where 'Contact ID' occurs exactly 3 times
mask = counts == 3

# Apply the mask to the DataFrame
df_filtered = df_duplicates[mask]

print(df_filtered)



In [ ]:
# Find all sets of records that are duplicate, drop them
df_working = df_working[~mask_duplicateRows]
print(df_working)

# Write to a file
df_working.to_csv(dir_transform + fileout_workingStep01, index=False)

In [ ]:
# Process the duplicates, removing all but the first row
df_duplicates_deduped_onFirst = df_duplicates.drop_duplicates(keep='first')
print(df_duplicates_deduped_onFirst)

In [ ]:
# SCRATCH
df_duplicates_deduped_onFirst.to_csv(dir_transform + "XXX")

In [ ]:
# these can then be added back to the working dataframe because they are now unique records
df_working = pd.concat([df_working, df_duplicates_deduped_onFirst])
df_working.sort_values(by='Contact ID', inplace=True)
print(df_working)

In [ ]:
# Write to file
df_working.to_csv(dir_transform + fileout_workingStep02)

In [ ]:
# Now need to deal with issue where there exist sets of identical Contact ID where other fields are different e.g., Primary Email.
# need to make sure we don't include sets where Contact ID is null/NaN
# because that could end up including different people whose records simply have null/NaN Contact ID in error
# those will be caught later

mask_manualCleanupDupeOnContactID = df_working['Contact ID'].duplicated(keep=False) & df_working['Contact ID'].notnull()
df_manualCleanupDupeOnContactID = df_working[mask_manualCleanupDupeOnContactID]
df_manualCleanupDupeOnContactID = df_manualCleanupDupeOnContactID.sort_values(by='Contact ID')
print(df_manualCleanupDupeOnContactID)

# Provide this file to staff for cleanup in the Civi source.
df_manualCleanupDupeOnContactID.to_csv(dir_transform + fileout_manualCleanupDupeOnContactID, index=False)

In [ ]:
df_working = df_working[~mask_manualCleanupDupeOnContactID]
df_working.sort_values(by='Contact ID', inplace=True)
print(df_working)

In [ ]:
# Now need to deal with records where primary email address (Primary Email) are duplicate but Contact ID is different
# is this also where we capture Contact ID is NaN/null?

mask_manualCleanupDupeOnEmail = df_working.duplicated(subset='Primary Email', keep=False)
df_manualCleanupDupeOnEmail = df_working[mask_manualCleanupDupeOnEmail]
df_manualCleanupDupeOnEmail = df_manualCleanupDupeOnEmail.sort_values(by='Primary Email')
print(df_manualCleanupDupeOnEmail)

# Provide this file to staff for cleanup in the Civi source.
df_manualCleanupDupeOnEmail.to_csv(dir_transform + fileout_manualCleanupDupeOnEmail, index=False)

In [ ]:
#df_working.drop_duplicates(subset="Primary Email",keep=False, inplace=True)
df_working = df_working[~mask_manualCleanupDupeOnEmail]
df_working.sort_values(by=['Contact ID'], inplace=True)
print(df_working)

In [ ]:
# Final check for record with blank/NaN Contact IDs that have not already been captured in searches for duplicates
mask_manualCleanupBlankContactID = df_working['Contact ID'].isna()
df_manualCleanupBlankContactID = df_working[mask_manualCleanupBlankContactID]
df_manualCleanupBlankContactID = df_manualCleanupBlankContactID.sort_values(by='Primary Email')
print(df_manualCleanupBlankContactID)

# Provide this file to staff for cleanup in the Civi source.
df_manualCleanupBlankContactID.to_csv(dir_transform + fileout_manualCleanupBlankContactID, index=False)

In [ ]:
df_working = df_working[~mask_manualCleanupBlankContactID]
df_working.sort_values(by=['Contact ID'], inplace=True)
print(df_working)